In [1]:
import folium # library for maping
import json
import pandas as pd
import os
from branca.colormap import LinearColormap
from bokeh.io import export_png

# Resources
# Boundaries of the zipcodes
phx_geo = "resources/phoenix_zip_codes_geo_from_us_az_simplified_ms.json"
# file containing the values to plot
val_file = os.path.join("resources","clean_sp.csv")
# file to which we want the map to be saved
date = "2018-03"
map_file = os.path.join("plots","zillow_sp"+date) #we will add the .html and .png extension
colorbar_caption = 'Sales Price '+date 
value_clabel = date
zipcode_clabel = 'ZipCode'

In [2]:
# Loading the Property values
df = pd.read_csv(val_file)
df = df.loc[:, [zipcode_clabel, value_clabel]]
df[zipcode_clabel] = df[zipcode_clabel].astype(str)
# 

# lets try to sort the values
df = df.dropna() # let's just try to remove the NaN rows
print(df.count())
# Transforming the data into a dictionnary (to be used by folium.GeoJson)
# One Entry by zipcode and the correspondig Zhvi value
map_dict = df.set_index(zipcode_clabel)[value_clabel].to_dict()
max(map_dict.values())


color_scale = LinearColormap(['yellow','red'], vmin = min(map_dict.values()), vmax =max(map_dict.values())/2)

ZipCode    132
2018-03    132
dtype: int64


In [5]:
# defining the function that will look into the color_scale to pick the color corresponding to the value
def get_color(feature):
    value = map_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

# Creating the map
m = folium.Map(location = [33.4484, -112.0740], zoom_start=8.5)
# Adding the colored zipcodes
folium.GeoJson(
    data = phx_geo,
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    }    
).add_to(m)
# Adding the colorbar
color_scale.caption = colorbar_caption
m.add_child(color_scale)

In [4]:
# Saving the map into an html file
# m.save(outfile = map_file+".html")
m.save(map_file)

In [ ]:
# import os
# import subprocess
# outdir = "plots" # this directory has to exist..

# url = "file://{}/tmp.html".format(os.getcwd())
# print(url)
# # outfn = os.path.join(outdir,"outfig.png")
# # print(outfn)
# outfn = "bla.png"
# subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])



# from GrabzIt import GrabzItClient

In [ ]:
# from config import app_key, app_secret

# app_key = "YTQzNjY5NTBlYjI0NDQzYmEwYjI2NTI4YmViNzYwNWM="
# app_secret = "JV8XFhs/Rj8/eD8/P3cQU0wiPz9PRg0/XCU/Pz8/Pz8="

# grabzIt = GrabzItClient.GrabzItClient(app_key, app_secret)
# grabzIt.HTMLToImage(map_file) 
# grabzIt.SaveTo("porra.jpg") 

# import time

# from bokeh.io import export_png

# export_png(m, filename="plot.png")


# from selenium import webdriver

# delay=5
# fn='testmap.html'
# tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
# m.save(fn)

# browser = webdriver.Firefox()
# browser.get(tmpurl)
# #Give the map tiles some time to load
# time.sleep(delay)
# browser.save_screenshot('map.png')
# browser.quit()